# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.
%load_ext dotenv
%dotenv 
#


In [2]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [3]:
import os
from glob import glob

# Write your code below.
parquet_files_path = glob(os.path.join(os.getenv("PRICE_DATA"),"*/*/*.parquet"))
parquet_files_path

['../../05_src/data/prices\\ACN\\ACN_2001\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2001\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2002\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2002\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2003\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2003\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2004\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2004\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2005\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2005\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2006\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2006\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2007\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2007\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2008\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2008\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_200

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [ ]:
# Write your code below.
dd_px = dd.read_parquet(parquet_files_path).set_index("ticker")

dd_shift = dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1),Adj_Close_lag_1 = x['Adj Close'].shift(1))
)
dd_rets = dd_shift.assign(
    returns = lambda x: x['Close']/x['Close_lag_1'] - 1
)
dd_hi_lo_range = dd_rets.assign(
    hi_lo_range = (dd_rets['High'] - dd_rets['Low'])
)

dd_feat=dd_hi_lo_range.copy()

C:\Users\kirti\AppData\Local\Temp\ipykernel_2028\558381904.py:4: UserWarning: `meta` is not specified, inferred from partial data.
Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result

  dd_shift = dd_px.groupby('ticker', group_keys=False).apply(


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [ ]:
# Write your code below.
df=dd_feat.compute()
df.dropna(inplace=True)
df["returns"]=df["returns"].astype(float)
df.reset_index(inplace=True)
df['rolling_10d'] = df.groupby('ticker')['returns'].transform(lambda x: x.rolling(10).mean())




,ticker,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,returns,hi_lo_range,rolling_10d
0,ACN,2018-07-05,163.699997,164.850006,163.160004,164.800003,160.651886,2427900.0,ACN.csv,2018,163.339996,0.008938,1.690002,NaN
1,ACN,2018-07-06,164.770004,165.169998,163.009995,164.509995,160.369171,1840400.0,ACN.csv,2018,164.800003,-0.001760,2.160004,NaN
2,ACN,2018-07-09,165.000000,166.259995,165.000000,166.130005,161.948410,1651600.0,ACN.csv,2018,164.509995,0.009847,1.259995,NaN
3,ACN,2018-07-10,166.500000,167.770004,166.339996,166.460007,162.270111,2071000.0,ACN.csv,2018,166.130005,0.001986,1.430008,NaN
4,ACN,2018-07-11,165.399994,167.100006,164.610001,166.309998,162.123886,1307100.0,ACN.csv,2018,166.460007,-0.000901,2.490005,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4699,ACN,2018-06-26,156.529999,157.589996,156.160004,156.860001,152.911743,2164000.0,ACN.csv,2018,156.160004,0.004483,1.429993,-0.003704
4700,ACN,2018-06-27,157.160004,158.699997,155.300003,155.339996,151.429993,3191900.0,ACN.csv,2018,156.860001,-0.009690,3.399994,-0.004783
4701,ACN,2018-06-28,160.490005,165.559998,159.699997,164.500000,160.359436,5801100.0,ACN.csv,2018,155.339996,0.058967,5.860001,0.001273
4702,ACN,2018-06-29,164.190002,165.559998,163.029999,163.589996,159.472321,3370000.0,ACN.csv,2018,164.500000,-0.005532,2.529999,0.000277


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
Yes, dask splits the data into samller chunks and rolling average would need the data rows to be arranged. Pandas is simpler and gives access to look at the data frame and work through if there any issues.
+ Would it have been better to do it in Dask? Why?
No,it would not as using dask would make the process complicated to execute. 

(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.